# Exercise 1: A* Motion Planning

In [ ]:
# The autoreload extension will automatically load in new code as you edit files, 
# so you don't need to restart the kernel every time
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate
from a_star import AStar, DetOccupancyGrid2D

For the A* motion planning problem we will start with a simple 2D environment with a set of obstacles defined below.

In [ ]:
width = 10
height = 10
occupancy = DetOccupancyGrid2D(width, height)
obstacles = [((6,7),(8,8)),((2,2),(4,3)),((2,5),(4,7)),((6,3),(8,5))]
occupancy.set_obstacles(obstacles)
occupancy.plot()

#### Exercise 1.1: Implement A* Algorithm and Solve a Simple 2D Motion Planning Problem
First, in the file `a_star.py` implement the functions:
1. `is_free`: a function to check if a given state is free (in the map and not in collision with an obstacle).
2. `distance`: the Euclidean distance between two states
3. `get_neighbors`: get the free neighbors from a given state
4. `solve`: solve the motion planning problem using A*

Run the code below to solve the simple 2D problem defined by `occupancy` above.

In [ ]:
x_init = (1, 9)
x_goal = (9, 1)
astar = AStar((0, 0), (width, height), x_init, x_goal, occupancy)
if not astar.solve():
    print("No path found")
else:
    astar.plot_path()
    astar.plot_tree()

#### Exercise 1.2: Motion Planning in a Random Cluttered Environment
Now we will run the code below to see our A* implementation run in a randomly generated cluttered environment.

In [ ]:
# Specify dimensions of the workspace, and some specifications for obstacle generation
width = 10
height = 10
occupancy = DetOccupancyGrid2D(width, height)
num_obs = 25
min_size = .5
max_size = 3
occupancy.generate_random_obstacles(num_obs, min_size, max_size)
x_init, x_goal = occupancy.generate_random_init_and_goal()

# Solve the problem using A*
astar = AStar((0, 0), (width, height), x_init, x_goal, occupancy, resolution=0.1)
if not astar.solve():
    print("No path found! (This is normal, try re-running.)")
else:
    astar.plot_path(fig_size=10)
    astar.plot_tree(point_size=2)

#### Exercise 1.3: Smooth A* Path
The A* path is built on a discrete grid and is therefore not a smooth path. We can smooth the path using splines, which may be more practically useful for robot path planning.

In the function `compute_smooth_plan`, implement the function to compute `scipy.interpolate.BSpline` objects from the A* path.

In [ ]:
def compute_smooth_plan(path, spline_alpha=0.05) -> np.ndarray:
    """
    Compute a smoothed path from the given A* path.
    """
    
    # Ensure path is a numpy array
    path = np.asarray(astar.path)

    # Compute and set the following variables:
    #   1. s: 
    #      Compute an array of arclengths, s, for each planned waypoint based on
    #      the Euclidean distance between points.
    #
    #   2. path_x_spline, path_y_spline (scipy.interpolate.BSpline):
    #      Fit cubic splines to the x and y coordinates of the path separately
    #      with respect to the computed time stamp array.
    
    ##### YOUR CODE STARTS HERE #####
    # Hint: Use scipy.interpolate.make_splrep and use `spline_alpha`

    ###### YOUR CODE END HERE ######

    # Compute smooth path from the interpolation
    x_smooth = scipy.interpolate.splev(s, path_x_spline, der=0)
    y_smooth = scipy.interpolate.splev(s, path_y_spline, der=0)
    return np.column_stack((x_smooth, y_smooth))

In [ ]:
# Compute smooth plan from A* path
smooth_plan = compute_smooth_plan(astar.path)

# Plot A* path and the smooth path
astar.plot_path(fig_size=10)
plt.plot(smooth_plan[:, 0], smooth_plan[:, 1], 'r-', label='Smoothed Path')
plt.legend()
plt.title('Path Smoothing')